In [1]:
# =========================================================
# Digital Image Processing Project: Restore Full Degraded Artwork Dataset
# =========================================================

import cv2
import numpy as np
import os
import time
from tqdm import tqdm  # Progress bar

# 1. Helper Functions
# =========================================================

def remove_noise_median(image):
    """Remove salt and pepper noise using median filter."""
    return cv2.medianBlur(image, 3)

def light_sharpen(image):
    """Apply light sharpening to restore details."""
    kernel = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]])
    return cv2.filter2D(image, -1, kernel)

def enhance_contrast_clahe(image):
    """Enhance contrast using CLAHE."""
    lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=1.5, tileGridSize=(8,8))
    cl = clahe.apply(l)
    merged = cv2.merge((cl, a, b))
    return cv2.cvtColor(merged, cv2.COLOR_LAB2RGB)

def gamma_correction(image, gamma=1.05):
    """Apply gamma correction."""
    invGamma = 1.0 / gamma
    table = np.array([(i / 255.0) ** invGamma * 255 for i in np.arange(256)]).astype("uint8")
    return cv2.LUT(image, table)

# 2. Full Restoration Pipeline on One Image
# =========================================================

def restore_artwork_single(img):
    """Restore a single degraded image."""
    step1 = remove_noise_median(img)
    step2 = light_sharpen(step1)
    step3 = enhance_contrast_clahe(step2)
    final_result = gamma_correction(step3)
    return final_result

# 3. Apply Restoration to Entire Dataset
# =========================================================

def restore_dataset(degraded_folder, restored_folder):
    """Restore all images in the degraded folder and save to restored folder."""
    if not os.path.exists(restored_folder):
        os.makedirs(restored_folder)

    image_files = [f for f in os.listdir(degraded_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    if not image_files:
        print(f"❌ No images found in: {degraded_folder}")
        return

    print(f"📂 Found {len(image_files)} degraded images to restore.")

    start_time = time.time()

    for filename in tqdm(image_files, desc="Restoring images", unit="image"):
        degraded_path = os.path.join(degraded_folder, filename)
        restored_path = os.path.join(restored_folder, filename)

        # Load degraded image
        img = cv2.imread(degraded_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Restore
        restored_img = restore_artwork_single(img)

        # Save restored image
        cv2.imwrite(restored_path, cv2.cvtColor(restored_img, cv2.COLOR_RGB2BGR))

    elapsed_time = time.time() - start_time
    print(f"✅ All images restored and saved to: {restored_folder}")
    print(f"🕒 Total processing time: {elapsed_time:.2f} seconds")

# 4. Example Run
# =========================================================

# Paths
degraded_folder = r"C:\Users\fatom\Downloads\IP_Dataset\degraded_full"   # Your degraded images folder
restored_folder = r"C:\Users\fatom\Downloads\IP_Dataset\restored_full"   # New folder for restored images

# Run restoration
restore_dataset(degraded_folder, restored_folder)


📂 Found 8683 degraded images to restore.


Restoring images: 100%|██████████| 8683/8683 [04:24<00:00, 32.87image/s]

✅ All images restored and saved to: C:\Users\fatom\Downloads\IP_Dataset\restored_full
🕒 Total processing time: 264.22 seconds
